In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
import os
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from google.colab import drive
drive.mount('/content/drive')

In [2]:
import os

# List the contents of a specific folder in your Drive, for example, a 'datasets' folder
drive_path = "train"

if os.path.exists(drive_path):
    files = os.listdir(drive_path)
    print(f"Files in {drive_path}:")
    for file in files:
        print(file)
else:
    print(f"The directory {drive_path} does not exist.")

Files in train:
leaf_blast
Rice Hispa
Neck_Blast
.DS_Store
Sheath Blight
healthy
bacterial_leaf_blight
brown_spot
leaf_scald
Tungro
narrow_brown_spot


In [3]:
model=models.resnet34(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)


# Freeze all layers except the final classification layer
#for name, param in model.named_parameters():
#    if 'fc' in name: # Check if the parameter belongs to the final fully connected layer
#        param.requires_grad = True
#    else:
#        param.requires_grad = False

/Users/giradasaiteja/env/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/giradasaiteja/env/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
from torchvision import transforms

# Define transformations for training data
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Define transformations for testing data
test_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [5]:
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
import torch

# Assume train_transforms is already defined

# 1. Define the path to your data directory
train_dir = 'train' # Corrected path

# 2. Create the full ImageFolder dataset
full_dataset = ImageFolder(root=train_dir, transform=train_transforms)
print(f"Total images in the dataset: {len(full_dataset)}")

# 3. Define the sizes for your training and validation sets
# Let's use an 80/20 split of the entire dataset
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
print(f"Splitting into {train_size} training samples and {val_size} validation samples.")


# 4. Perform the split using random_split
# This ensures the validation set has not been seen during training
# A generator is used for reproducibility of the split
generator = torch.Generator().manual_seed(42) # Use a fixed seed for consistent splits
train_subset, val_subset = random_split(full_dataset, [train_size, val_size], generator=generator)

# 5. Create the DataLoaders from the new Subset objects
# Use clear and distinct variable names
train_loader = DataLoader(
    train_subset,
    batch_size=32,
    shuffle=True,
    num_workers=2, # Reduced number of workers
    pin_memory=True
)

val_loader = DataLoader(
    val_subset,
    batch_size=32,
    shuffle=False,  # No need to shuffle the validation set
    num_workers=2, # Reduced number of workers
    pin_memory=True
)

print("DataLoaders for training and validation have been created successfully.")

Total images in the dataset: 1089
Splitting into 871 training samples and 218 validation samples.
DataLoaders for training and validation have been created successfully.


In [6]:
criterian=nn.CrossEntropyLoss()

In [19]:
import torch.optim as optim
loss_fn=nn.CrossEntropyLoss()
# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [20]:

def train(model,train_loader,loss_fn,optimizer):
    model.train()

    running_loss=0.0
    correc_predict=0
    total_samp=0
    for inputs,labels in train_loader:
        inputs,labels=inputs.to(device),labels.to(device)
        optimizer.zero_grad()
        outputs=model(inputs)
        loss=loss_fn(outputs,labels)
        loss.backward()
        optimizer.step() # Use xm.optimizer_step for TPU # Add mark_step for TPU


        running_loss=loss.item()*inputs.size(0)
        _,predicted=torch.max(outputs.data,1)
        total_samp+=labels.size(0)
        correc_predict+=(predicted==labels).sum().item()
    epoch_loss=running_loss/total_samp
    epoch_acc=100*correc_predict/total_samp

    return epoch_loss,epoch_acc

def evaluate(model,val_loader,loss_fn):
    model.eval()

    running_loss=0.0
    correc_predict=0
    total_samp=0
    with torch.no_grad():
        for inputs,labels in val_loader:
            inputs,labels=inputs.to(device),labels.to(device)

            outputs=model(inputs)
            loss=loss_fn(outputs,labels)

            running_loss=loss.item()*inputs.size(0)
        _,predicted=torch.max(outputs.data,1)
        total_samp+=labels.size(0)
        correc_predict+=(predicted==labels).sum().item()
    epoch_loss=running_loss/total_samp
    epoch_acc=100*correc_predict/total_samp

    return epoch_loss,epoch_acc

**Reasoning**:
I will execute the training loop for a specified number of epochs and evaluate the model periodically. Since the previous steps for setting up the training loop and evaluation are complete, I can combine these into a single block to train and evaluate the model for a few epochs. I will set the number of epochs to 5 and evaluate after each epoch to monitor the training progress.



In [21]:
eproaches=12
model.to(device)
for epoch in range(eproaches):
    train_loss,train_acc=train(model,train_loader,loss_fn,optimizer)
    eval_loss,eval_acc=evaluate(model,val_loader,loss_fn)

    print(f"epoch{epoch+1}:"
          f"train_loss:{train_loss:.2f},train_acc{train_acc:.4f}:"
          f"eval_loss:{eval_loss:.2f},eval_acc{eval_acc:.4f}:")
print("training finished")

epoch1:train_loss:0.00,train_acc88.7486:eval_loss:0.63,eval_acc92.3077:
epoch2:train_loss:0.00,train_acc92.9966:eval_loss:0.47,eval_acc88.4615:
epoch3:train_loss:0.00,train_acc92.7669:eval_loss:0.56,eval_acc88.4615:
epoch4:train_loss:0.01,train_acc93.5706:eval_loss:0.24,eval_acc92.3077:
epoch5:train_loss:0.00,train_acc94.2595:eval_loss:0.25,eval_acc88.4615:
epoch6:train_loss:0.00,train_acc92.9966:eval_loss:0.33,eval_acc92.3077:
epoch7:train_loss:0.00,train_acc94.8335:eval_loss:0.39,eval_acc88.4615:
epoch8:train_loss:0.01,train_acc94.3743:eval_loss:0.27,eval_acc92.3077:
epoch9:train_loss:0.01,train_acc94.0299:eval_loss:0.34,eval_acc88.4615:
epoch10:train_loss:0.01,train_acc94.7187:eval_loss:0.51,eval_acc92.3077:
epoch11:train_loss:0.01,train_acc93.4558:eval_loss:0.42,eval_acc88.4615:
epoch12:train_loss:0.01,train_acc93.5706:eval_loss:0.44,eval_acc88.4615:
training finished


In [22]:
MODEL_SAVE_PATH = 'rice_leaf_disease.pth'

# Save the model's state dictionary
torch.save(model.state_dict(), MODEL_SAVE_PATH)

print(f"Model saved to {MODEL_SAVE_PATH}")

Model saved to rice_leaf_disease.pth


In [25]:
! python3 predict_custom.py rice_hispa1.jpg

Setting up your custom model and environment...
Using device: mps
predict_custom.py:69: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(MODEL_